<a href="https://colab.research.google.com/github/Karthik-Anguru/NLP-1/blob/main/ProEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TEXT

In [1]:
import pandas as pd
dataset =pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [3]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [4]:
for index , tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


Text Cleaning

In [5]:
import re
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9]',' ',text)
    text = re.sub(r'[^\x00-\x7F]+',' ',text)
    text = text.lower()
    return text

In [6]:
dataset['clean_text'] = dataset.tweet.apply(lambda x:clean_text(x))
dataset.head(10)


,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr...


FEATURE ENGINEERING

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
len(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [8]:
stop = stopwords.words('english')

In [9]:
def gen_freq(text):
    word_list = []
    for word in text.split():
        word_list.extend(word)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop,errors='ignore')
    return word_freq

In [10]:
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0


In [11]:
def any_neg(words):
    for word in words:
        if word in ['n','no','non','not'] or re.search(r"\wn't",word):
            return 1
        else:
            return 0

In [12]:
def is_question(words,ques_list):
    for word in words:
        if word in ['when','what','how','why','who','where']:
            return 1
        else:
            return 0

In [13]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 =word_freq[-100:]
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x:len(x))
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x:any_neg(x))
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x:is_question(x,is_question))
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x:any_rare(x,rare_100))
dataset['char_count'] = dataset.clean_text.apply(lambda x:len(x))


In [14]:
wl={'text':['hi','how','are','you','I','am','good','it','how','are','you']}
df=pd.DataFrame(wl)
df

,text
0,hi
1,how
2,are
3,you
4,I
5,am
6,good
7,it
8,how
9,are


In [16]:
from sklearn.model_selection import train_test_split
X= dataset[['word_count','any_neg','any_rare','char_count']]
y=dataset.label
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [18]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model=model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [19]:
model.predict(X_test[5:10])

array([1, 1, 1, 1, 1])

In [21]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,y_pred)*100)

Accuracy: 42.89799809342231


In [22]:
from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier()
clf_rf.fit(X_train,y_train)
y_pred_rf=clf_rf.predict(X_test).astype(int)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
accuracy_score
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test,y_pred_rf))
print("Accuracy:",accuracy_score(y_test,y_pred_rf))

[[395 204]
 [217 233]]
              precision    recall  f1-score   support

           0       0.65      0.66      0.65       599
           1       0.53      0.52      0.53       450

    accuracy                           0.60      1049
   macro avg       0.59      0.59      0.59      1049
weighted avg       0.60      0.60      0.60      1049

Accuracy: 0.5986653956148713


In [25]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(X_train,y_train)
y_pred_log=logreg.predict(X_test)

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_log))


              precision    recall  f1-score   support

           0       0.57      0.93      0.71       599
           1       0.41      0.07      0.11       450

    accuracy                           0.56      1049
   macro avg       0.49      0.50      0.41      1049
weighted avg       0.50      0.56      0.45      1049

